# Conditional GAN from Mlmastery

In [ ]:
# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate

# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1), n_classes=10):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# scale up to image dimensions with linear activation
	n_nodes = in_shape[0] * in_shape[1]
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((in_shape[0], in_shape[1], 1))(li)
	# image input
	in_image = Input(shape=in_shape)
	# concat label as a channel
	merge = Concatenate()([in_image, li])
	# downsample
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
	fe = LeakyReLU(alpha=0.2)(fe)
	# downsample
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	# flatten feature maps
	fe = Flatten()(fe)
	# dropout
	fe = Dropout(0.4)(fe)
	# output
	out_layer = Dense(1, activation='sigmoid')(fe)
	# define model
	model = Model([in_image, in_label], out_layer)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define the standalone generator model
def define_generator(latent_dim, n_classes=10):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# linear multiplication
	n_nodes = 7 * 7
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((7, 7, 1))(li)
	# image generator input
	in_lat = Input(shape=(latent_dim,))
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	gen = Dense(n_nodes)(in_lat)
	gen = LeakyReLU(alpha=0.2)(gen)
	gen = Reshape((7, 7, 128))(gen)
	# merge image gen and label input
	merge = Concatenate()([gen, li])
	# upsample to 14x14
	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 28x28
	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# output
	out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
	# define model
	model = Model([in_lat, in_label], out_layer)
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise, gen_label = g_model.input
	# get image output from the generator model
	gen_output = g_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = d_model([gen_output, gen_label])
	# define gan model as taking noise and label and outputting a classification
	model = Model([gen_noise, gen_label], gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# load fashion mnist images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) = load_data()
	# expand to 3d, e.g. add channels
	X = expand_dims(trainX, axis=-1)
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return [X, trainy]

# # select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	y = zeros((n_samples, 1))
	return [images, labels_input], y

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			[X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
			# generate 'fake' examples
			[X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
			# prepare points in latent space as input for the generator
			[z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
	# save the generator model
	g_model.save('cgan_generator.h5')

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

Using TensorFlow backend.
/home/nisnab/miniconda3/envs/tf_gpu_copy/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/nisnab/miniconda3/envs/tf_gpu_copy/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/nisnab/miniconda3/envs/tf_gpu_copy/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/nisnab/m

>1, 1/468, d1=0.676, d2=0.696 g=0.691
>1, 2/468, d1=0.601, d2=0.699 g=0.687
>1, 3/468, d1=0.539, d2=0.707 g=0.681
>1, 4/468, d1=0.479, d2=0.719 g=0.671
>1, 5/468, d1=0.411, d2=0.737 g=0.656
>1, 6/468, d1=0.370, d2=0.763 g=0.641
>1, 7/468, d1=0.337, d2=0.792 g=0.626
>1, 8/468, d1=0.307, d2=0.810 g=0.623
>1, 9/468, d1=0.264, d2=0.806 g=0.653


/home/nisnab/miniconda3/envs/tf_gpu_copy/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 10/468, d1=0.300, d2=0.757 g=0.700
>1, 11/468, d1=0.285, d2=0.698 g=0.783
>1, 12/468, d1=0.303, d2=0.624 g=0.858
>1, 13/468, d1=0.345, d2=0.589 g=0.882
>1, 14/468, d1=0.300, d2=0.596 g=0.849
>1, 15/468, d1=0.266, d2=0.630 g=0.808
>1, 16/468, d1=0.253, d2=0.677 g=0.753
>1, 17/468, d1=0.224, d2=0.726 g=0.707
>1, 18/468, d1=0.258, d2=0.771 g=0.684
>1, 19/468, d1=0.208, d2=0.776 g=0.688
>1, 20/468, d1=0.147, d2=0.779 g=0.693
>1, 21/468, d1=0.132, d2=0.803 g=0.686
>1, 22/468, d1=0.148, d2=0.827 g=0.683
>1, 23/468, d1=0.124, d2=0.808 g=0.735
>1, 24/468, d1=0.094, d2=0.700 g=0.853
>1, 25/468, d1=0.087, d2=0.562 g=1.004
>1, 26/468, d1=0.078, d2=0.489 g=1.080
>1, 27/468, d1=0.065, d2=0.504 g=1.033
>1, 28/468, d1=0.071, d2=0.545 g=0.913
>1, 29/468, d1=0.053, d2=0.616 g=0.817
>1, 30/468, d1=0.047, d2=0.682 g=0.742
>1, 31/468, d1=0.051, d2=0.750 g=0.696
>1, 32/468, d1=0.076, d2=0.789 g=0.652
>1, 33/468, d1=0.063, d2=0.825 g=0.631
>1, 34/468, d1=0.050, d2=0.837 g=0.628
>1, 35/468, d1=0.089, d2=

>1, 219/468, d1=0.780, d2=0.564 g=0.990
>1, 220/468, d1=0.712, d2=0.532 g=1.005
>1, 221/468, d1=0.711, d2=0.539 g=1.045
>1, 222/468, d1=0.692, d2=0.515 g=1.043
>1, 223/468, d1=0.645, d2=0.515 g=1.029
>1, 224/468, d1=0.609, d2=0.567 g=0.996
>1, 225/468, d1=0.675, d2=0.627 g=0.927
>1, 226/468, d1=0.645, d2=0.721 g=0.794
>1, 227/468, d1=0.616, d2=0.769 g=0.744
>1, 228/468, d1=0.670, d2=0.791 g=0.702
>1, 229/468, d1=0.663, d2=0.806 g=0.678
>1, 230/468, d1=0.719, d2=0.840 g=0.645
>1, 231/468, d1=0.747, d2=0.792 g=0.662
>1, 232/468, d1=0.773, d2=0.814 g=0.642
>1, 233/468, d1=0.776, d2=0.838 g=0.652
>1, 234/468, d1=0.784, d2=0.801 g=0.676
>1, 235/468, d1=0.788, d2=0.816 g=0.672
>1, 236/468, d1=0.777, d2=0.822 g=0.670
>1, 237/468, d1=0.797, d2=0.787 g=0.701
>1, 238/468, d1=0.786, d2=0.749 g=0.707
>1, 239/468, d1=0.758, d2=0.725 g=0.736
>1, 240/468, d1=0.760, d2=0.713 g=0.744
>1, 241/468, d1=0.784, d2=0.697 g=0.768
>1, 242/468, d1=0.765, d2=0.694 g=0.802
>1, 243/468, d1=0.760, d2=0.688 g=0.811


...

>5, 244/468, d1=0.673, d2=0.677 g=0.768
>5, 245/468, d1=0.650, d2=0.672 g=0.749
>5, 246/468, d1=0.689, d2=0.640 g=0.792
>5, 247/468, d1=0.634, d2=0.639 g=0.821
>5, 248/468, d1=0.693, d2=0.638 g=0.798
>5, 249/468, d1=0.700, d2=0.666 g=0.795
>5, 250/468, d1=0.700, d2=0.668 g=0.767
>5, 251/468, d1=0.650, d2=0.661 g=0.757
>5, 252/468, d1=0.657, d2=0.682 g=0.781
>5, 253/468, d1=0.687, d2=0.671 g=0.775
>5, 254/468, d1=0.661, d2=0.657 g=0.767
>5, 255/468, d1=0.653, d2=0.673 g=0.757
>5, 256/468, d1=0.637, d2=0.675 g=0.754
>5, 257/468, d1=0.651, d2=0.666 g=0.736
>5, 258/468, d1=0.640, d2=0.702 g=0.741
>5, 259/468, d1=0.601, d2=0.716 g=0.722
>5, 260/468, d1=0.649, d2=0.720 g=0.740
>5, 261/468, d1=0.660, d2=0.701 g=0.756
>5, 262/468, d1=0.660, d2=0.650 g=0.807
>5, 263/468, d1=0.680, d2=0.648 g=0.829
>5, 264/468, d1=0.659, d2=0.616 g=0.851
>5, 265/468, d1=0.690, d2=0.648 g=0.805
>5, 266/468, d1=0.718, d2=0.655 g=0.771
>5, 267/468, d1=0.658, d2=0.694 g=0.754
>5, 268/468, d1=0.677, d2=0.702 g=0.724


>5, 456/468, d1=0.647, d2=0.654 g=0.775
>5, 457/468, d1=0.658, d2=0.662 g=0.742
>5, 458/468, d1=0.682, d2=0.657 g=0.773
>5, 459/468, d1=0.638, d2=0.656 g=0.792
>5, 460/468, d1=0.652, d2=0.628 g=0.813
>5, 461/468, d1=0.660, d2=0.649 g=0.803
>5, 462/468, d1=0.650, d2=0.698 g=0.776
>5, 463/468, d1=0.674, d2=0.661 g=0.768
>5, 464/468, d1=0.636, d2=0.692 g=0.750
>5, 465/468, d1=0.665, d2=0.676 g=0.770
>5, 466/468, d1=0.633, d2=0.666 g=0.796
>5, 467/468, d1=0.656, d2=0.653 g=0.778
>5, 468/468, d1=0.641, d2=0.649 g=0.807
>6, 1/468, d1=0.637, d2=0.639 g=0.818
>6, 2/468, d1=0.685, d2=0.638 g=0.821
>6, 3/468, d1=0.679, d2=0.601 g=0.797
>6, 4/468, d1=0.666, d2=0.644 g=0.797
>6, 5/468, d1=0.683, d2=0.650 g=0.792
>6, 6/468, d1=0.654, d2=0.665 g=0.790
>6, 7/468, d1=0.649, d2=0.659 g=0.787
>6, 8/468, d1=0.636, d2=0.637 g=0.793
>6, 9/468, d1=0.654, d2=0.670 g=0.802
>6, 10/468, d1=0.671, d2=0.673 g=0.766
>6, 11/468, d1=0.658, d2=0.639 g=0.794
>6, 12/468, d1=0.682, d2=0.660 g=0.797
>6, 13/468, d1=0.660,

>6, 196/468, d1=0.656, d2=0.677 g=0.760
>6, 197/468, d1=0.616, d2=0.681 g=0.768
>6, 198/468, d1=0.654, d2=0.679 g=0.766
>6, 199/468, d1=0.634, d2=0.652 g=0.831
>6, 200/468, d1=0.660, d2=0.626 g=0.844
>6, 201/468, d1=0.638, d2=0.650 g=0.814
>6, 202/468, d1=0.624, d2=0.600 g=0.800
>6, 203/468, d1=0.681, d2=0.666 g=0.802
>6, 204/468, d1=0.683, d2=0.649 g=0.782
>6, 205/468, d1=0.679, d2=0.671 g=0.782
>6, 206/468, d1=0.676, d2=0.656 g=0.815
>6, 207/468, d1=0.671, d2=0.624 g=0.832
>6, 208/468, d1=0.614, d2=0.619 g=0.818
>6, 209/468, d1=0.635, d2=0.624 g=0.780
>6, 210/468, d1=0.669, d2=0.644 g=0.791
>6, 211/468, d1=0.619, d2=0.745 g=0.742
>6, 212/468, d1=0.614, d2=0.670 g=0.775
>6, 213/468, d1=0.658, d2=0.660 g=0.822
>6, 214/468, d1=0.670, d2=0.601 g=0.888
>6, 215/468, d1=0.651, d2=0.567 g=0.915
>6, 216/468, d1=0.689, d2=0.608 g=0.844
>6, 217/468, d1=0.684, d2=0.657 g=0.808
>6, 218/468, d1=0.633, d2=0.653 g=0.786
>6, 219/468, d1=0.654, d2=0.690 g=0.744
>6, 220/468, d1=0.628, d2=0.660 g=0.776


>6, 401/468, d1=0.732, d2=0.687 g=0.796
>6, 402/468, d1=0.643, d2=0.713 g=0.749
>6, 403/468, d1=0.710, d2=0.691 g=0.788
>6, 404/468, d1=0.656, d2=0.638 g=0.828
>6, 405/468, d1=0.629, d2=0.607 g=0.864
>6, 406/468, d1=0.655, d2=0.594 g=0.857
>6, 407/468, d1=0.675, d2=0.635 g=0.814
>6, 408/468, d1=0.615, d2=0.690 g=0.756
>6, 409/468, d1=0.642, d2=0.733 g=0.746
>6, 410/468, d1=0.637, d2=0.686 g=0.735
>6, 411/468, d1=0.632, d2=0.721 g=0.764
>6, 412/468, d1=0.645, d2=0.661 g=0.837
>6, 413/468, d1=0.688, d2=0.579 g=0.921
>6, 414/468, d1=0.677, d2=0.560 g=0.920
>6, 415/468, d1=0.721, d2=0.604 g=0.890
>6, 416/468, d1=0.694, d2=0.638 g=0.830
>6, 417/468, d1=0.681, d2=0.651 g=0.788
>6, 418/468, d1=0.670, d2=0.714 g=0.730
>6, 419/468, d1=0.644, d2=0.726 g=0.720
>6, 420/468, d1=0.658, d2=0.684 g=0.784
>6, 421/468, d1=0.637, d2=0.631 g=0.811
>6, 422/468, d1=0.625, d2=0.628 g=0.865
>6, 423/468, d1=0.663, d2=0.646 g=0.860
>6, 424/468, d1=0.615, d2=0.677 g=0.825
>6, 425/468, d1=0.660, d2=0.685 g=0.764


>7, 141/468, d1=0.684, d2=0.667 g=0.774
>7, 142/468, d1=0.672, d2=0.668 g=0.788
>7, 143/468, d1=0.637, d2=0.672 g=0.776
>7, 144/468, d1=0.673, d2=0.643 g=0.766
>7, 145/468, d1=0.686, d2=0.658 g=0.772
>7, 146/468, d1=0.661, d2=0.667 g=0.755
>7, 147/468, d1=0.670, d2=0.642 g=0.782
>7, 148/468, d1=0.649, d2=0.668 g=0.767
>7, 149/468, d1=0.678, d2=0.667 g=0.738
>7, 150/468, d1=0.645, d2=0.646 g=0.789
>7, 151/468, d1=0.637, d2=0.656 g=0.798
>7, 152/468, d1=0.642, d2=0.652 g=0.796
>7, 153/468, d1=0.678, d2=0.657 g=0.771
>7, 154/468, d1=0.675, d2=0.738 g=0.741
>7, 155/468, d1=0.651, d2=0.662 g=0.748
>7, 156/468, d1=0.662, d2=0.653 g=0.794
>7, 157/468, d1=0.674, d2=0.691 g=0.784
>7, 158/468, d1=0.653, d2=0.645 g=0.799
>7, 159/468, d1=0.674, d2=0.629 g=0.792
>7, 160/468, d1=0.669, d2=0.664 g=0.769
>7, 161/468, d1=0.656, d2=0.661 g=0.786
>7, 162/468, d1=0.639, d2=0.647 g=0.774
>7, 163/468, d1=0.699, d2=0.663 g=0.766
>7, 164/468, d1=0.651, d2=0.676 g=0.787
>7, 165/468, d1=0.670, d2=0.674 g=0.798


# Conditional GAN

In [1]:
# Simple example of conditional GAN in Keras
# Generates MNIST numbers of one's choice, not at random as in standard GANs
# 
# author: Alejandro Pozas-Kerstjens
#
# Note: tricks displayed refer to those mentioned in https://github.com/soumith/ganhacks

import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import matplotlib.pyplot as plt
from keras.layers import LeakyReLU, Activation, Input, Dense, Dropout, Concatenate, BatchNormalization
from keras.models import Model
from keras.regularizers import l1_l2
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical
from keras.datasets import mnist

def build_gan(generator, discriminator, name="gan"):
    '''Build the GAN from a generator and a discriminator'''
    yfake = Activation("linear", name="yfake")(discriminator(generator(generator.inputs)))
    yreal = Activation("linear", name="yreal")(discriminator(discriminator.inputs))
    model = Model(generator.inputs + discriminator.inputs, [yfake, yreal], name=name)
    return model
    

def disc(image_dim, label_dim, layer_dim=1024, reg=lambda: l1_l2(1e-5, 1e-5)):
    '''Discriminator network'''
    x      = (Input(shape=(image_dim,), name='discriminator_input'))
    label  = (Input(shape=(label_dim,), name='discriminator_label'))
    inputs = (Concatenate(name='input_concatenation'))([x, label])
    a = (Dense(layer_dim, name="discriminator_h1", kernel_regularizer=reg()))(inputs)
    a = (LeakyReLU(0.2))(a)
    a = (Dense(int(layer_dim / 2), name="discriminator_h2", kernel_regularizer=reg()))(a)
    a = (LeakyReLU(0.2))(a)
    a = (Dense(int(layer_dim / 4), name="discriminator_h3", kernel_regularizer=reg()))(a)
    a = (LeakyReLU(0.2))(a)
    a = (Dense(1, name="discriminator_y", kernel_regularizer=reg()))(a)
    a = (Activation('sigmoid'))(a)
    model = Model(inputs=[x, label], outputs=a, name="discriminator")
    return model
    
    
def gen(noise_dim, label_dim, image_dim, layer_dim=1024, activ='tanh', reg=lambda: l1_l2(1e-5, 1e-5)):
    '''Generator network'''
    z      = (Input(shape=(noise_dim,), name='generator_input'))
    label  = (Input(shape=(label_dim,), name='generator_label'))
    inputs = (Concatenate(name='input_concatenation'))([z, label])
    a = (Dense(int(layer_dim / 4), name="generator_h1", kernel_regularizer=reg()))(inputs)
    a = (LeakyReLU(0.2))(a)    # Trick 5
    a = (Dense(int(layer_dim / 2), name="generator_h2", kernel_regularizer=reg()))(a)
    a = (LeakyReLU(0.2))(a)
    a = (Dense(layer_dim, name="generator_h3", kernel_regularizer=reg()))(a)
    a = (LeakyReLU(0.2))(a)
    a = (Dense(np.prod(image_dim), name="generator_x_flat", kernel_regularizer=reg()))(a)
    a = (Activation(activ))(a)    
    model = Model(inputs=[z, label], outputs=[a, label], name="generator")
    return model

    
def make_trainable(net, val):
    '''Changes the trainable property of a model as a whole and layer by layer'''
    net.trainable = val
    for l in net.layers:
        l.trainable = val
    

# ------------------------------------------------------------------------------
# Data preparation
# ------------------------------------------------------------------------------
(x_train, l_train), (x_test, l_test) = mnist.load_data()
x_train = np.concatenate((x_train, x_test))
l_train = np.concatenate((l_train, l_test))

# Normalization according to Trick 1
x_train = x_train.reshape(x_train.shape[0], 784)
x_train = (x_train - 127.5) / 127.5
l_train = to_categorical(l_train)


# ------------------------------------------------------------------------------
# Parameter choice
# ------------------------------------------------------------------------------    
# Dimension of noise to be fed to the generator
noise_dim = 100
# Dimension of images generated
image_dim = 28 * 28
# Dimension of labels
label_dim = 10

batch_size  = 100
num_batches = int(x_train.shape[0] / batch_size)
num_epochs  = 30

# ------------------------------------------------------------------------------
# Network creation
# ------------------------------------------------------------------------------
# Create generator ((z, l) -> (x, l))
generator = gen(noise_dim, label_dim, image_dim)
adam = Adam(lr=0.0002, beta_1=0.5)
generator.compile(loss='binary_crossentropy', optimizer=adam)    # Trick 9

# Create discriminator ((x, l) -> y)
discriminator = disc(image_dim, label_dim)
discriminator.compile(loss='binary_crossentropy', optimizer='SGD')    # Trick 9

# Build GAN. Note how the discriminator is set to be not trainable since the beginning
make_trainable(discriminator, False)
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=adam)


# ------------------------------------------------------------------------------
# Training
# ------------------------------------------------------------------------------
for epoch in range(num_epochs):
    print("Epoch {}/{}".format(epoch + 1, num_epochs))
    for index in range(num_batches):
        # Train the discriminator. It looks like training works best if it is trained first on only real data, and then only
        # on fake data, so let's do that. This is Trick 4.
        make_trainable(discriminator, True)
        # Train dicriminator on real data
        batch       = np.random.randint(0, x_train.shape[0], size=batch_size)
        image_batch = x_train[batch]
        label_batch = l_train[batch]
        y_real      = np.ones(batch_size) + 0.2 * np.random.uniform(-1, 1, size=batch_size)    # Label smoothing. Trick 6
        discriminator.train_on_batch([image_batch, label_batch], y_real)
        # Train the discriminator on fake data
        noise_batch      = np.random.normal(0, 1, (batch_size, noise_dim))    # Trick 3
        generated_images = generator.predict([noise_batch, label_batch])
        y_fake           = np.zeros(batch_size) + 0.2 * np.random.uniform(0, 1, size=batch_size)    # Label smoothing
        d_loss = discriminator.train_on_batch(generated_images, y_fake)   # Recall that generated_images already contains the labels
        # Train the generator. We train it through the whole model. There is a very subtle point here. We want to minimize the error
        # of the discriminator, but on the other hand we want to have the generator maximizing the loss of the discriminator (make him
        # not capable of distinguishing which images are real). One way to achieve this is to change the loss function of the generator
        # by some kind of "negative loss", which in practice is implemented by switching the labels of the real and the fake
        # images. Note that when training the discriminator we were doing the assignment real_image->1, fake_image->0, so now
        # we will do real_image->0, fake_image->1. The order of the outputs is [fake, real], as given by build_gan(). This is Trick 2.
        make_trainable(discriminator, False)
        gan_loss = gan.train_on_batch([noise_batch, label_batch, image_batch, label_batch], [y_real, y_fake])

        print(
            "Batch {}/{}: Discriminator loss = {}, GAN loss = {}".format(index + 1, num_batches, d_loss,
                                                                         gan_loss))
# Save weights. Just saving the whole GAN should work as well
generator.save_weights('generator_cGAN.h5')
discriminator.save_weights('discriminator_cGAN.h5')
gan.save_weights('gan_cGAN.h5')

Using TensorFlow backend.


Epoch 1/30
Batch 1/700: Discriminator loss = 1.2148332595825195, GAN loss = [3.7418015, 0.6616232, 2.1143382]
Batch 2/700: Discriminator loss = 1.3512165546417236, GAN loss = [3.9408474, 0.53972405, 2.4353607]
Batch 3/700: Discriminator loss = 1.508239984512329, GAN loss = [4.052915, 0.44131073, 2.6459115]
Batch 4/700: Discriminator loss = 1.6210124492645264, GAN loss = [3.9980297, 0.39462417, 2.637777]
Batch 5/700: Discriminator loss = 1.7666152715682983, GAN loss = [3.9609337, 0.337075, 2.658297]
Batch 6/700: Discriminator loss = 1.8829948902130127, GAN loss = [3.7752619, 0.34087598, 2.4688911]
Batch 7/700: Discriminator loss = 1.9254709482192993, GAN loss = [3.6310256, 0.32476532, 2.3408394]
Batch 8/700: Discriminator loss = 1.9242677688598633, GAN loss = [3.524834, 0.3486791, 2.2108064]
Batch 9/700: Discriminator loss = 1.8817198276519775, GAN loss = [3.483163, 0.35155046, 2.166337]
Batch 10/700: Discriminator loss = 1.8492584228515625, GAN loss = [3.427281, 0.41143924, 2.050636]
B

Batch 86/700: Discriminator loss = 1.6338741779327393, GAN loss = [2.5123472, 0.8232923, 0.7301072]
Batch 87/700: Discriminator loss = 1.611010193824768, GAN loss = [2.4967158, 0.79617393, 0.74166465]
Batch 88/700: Discriminator loss = 1.5930440425872803, GAN loss = [2.4756355, 0.7781681, 0.73866075]
Batch 89/700: Discriminator loss = 1.543891429901123, GAN loss = [2.519611, 0.7786584, 0.7822164]
Batch 90/700: Discriminator loss = 1.5415540933609009, GAN loss = [2.5089064, 0.7955412, 0.7546991]
Batch 91/700: Discriminator loss = 1.499546766281128, GAN loss = [2.516527, 0.7883119, 0.7696199]
Batch 92/700: Discriminator loss = 1.5000752210617065, GAN loss = [2.4835505, 0.77092606, 0.7540987]
Batch 93/700: Discriminator loss = 1.4857004880905151, GAN loss = [2.494513, 0.77341753, 0.76263636]
Batch 94/700: Discriminator loss = 1.4723342657089233, GAN loss = [2.4967787, 0.78491104, 0.75347215]
Batch 95/700: Discriminator loss = 1.4576003551483154, GAN loss = [2.4798207, 0.78499115, 0.736497

Batch 169/700: Discriminator loss = 1.228506326675415, GAN loss = [2.9589615, 0.9983594, 1.0088559]
Batch 170/700: Discriminator loss = 1.2304705381393433, GAN loss = [2.9495656, 0.9767485, 1.0211856]
Batch 171/700: Discriminator loss = 1.2675700187683105, GAN loss = [2.852952, 0.96174395, 0.93968713]
Batch 172/700: Discriminator loss = 1.2603381872177124, GAN loss = [2.8410714, 0.93826175, 0.95139694]
Batch 173/700: Discriminator loss = 1.3416470289230347, GAN loss = [2.81355, 0.953985, 0.90825796]
Batch 174/700: Discriminator loss = 1.3296704292297363, GAN loss = [2.7762253, 0.91837955, 0.9066435]
Batch 175/700: Discriminator loss = 1.3750784397125244, GAN loss = [2.6961098, 0.89106154, 0.85394496]
Batch 176/700: Discriminator loss = 1.3909739255905151, GAN loss = [2.6184933, 0.83946854, 0.8280144]
Batch 177/700: Discriminator loss = 1.4678670167922974, GAN loss = [2.5662107, 0.83513826, 0.7801484]
Batch 178/700: Discriminator loss = 1.4528828859329224, GAN loss = [2.5295627, 0.80024

Batch 259/700: Discriminator loss = 1.287503957748413, GAN loss = [2.5844562, 0.8209103, 0.8192804]
Batch 260/700: Discriminator loss = 1.2705897092819214, GAN loss = [2.5781758, 0.8098037, 0.8241936]
Batch 261/700: Discriminator loss = 1.2745068073272705, GAN loss = [2.5638428, 0.80732286, 0.8124261]
Batch 262/700: Discriminator loss = 1.260688304901123, GAN loss = [2.5900123, 0.81239396, 0.833609]
Batch 263/700: Discriminator loss = 1.2604023218154907, GAN loss = [2.6020308, 0.81357706, 0.8445288]
Batch 264/700: Discriminator loss = 1.2647732496261597, GAN loss = [2.6019423, 0.82313365, 0.8349697]
Batch 265/700: Discriminator loss = 1.261710524559021, GAN loss = [2.6058621, 0.8259706, 0.83613986]
Batch 266/700: Discriminator loss = 1.2548887729644775, GAN loss = [2.5934165, 0.8248056, 0.8249481]
Batch 267/700: Discriminator loss = 1.2427923679351807, GAN loss = [2.6170099, 0.83148646, 0.84194964]
Batch 268/700: Discriminator loss = 1.2431411743164062, GAN loss = [2.6227334, 0.837491,

...

Batch 237/700: Discriminator loss = 1.0921216011047363, GAN loss = [2.6913881, 0.97452426, 1.0966299]
Batch 238/700: Discriminator loss = 1.112987995147705, GAN loss = [2.6931083, 0.9755458, 1.0973095]
Batch 239/700: Discriminator loss = 1.0990557670593262, GAN loss = [2.6418386, 0.9736589, 1.0479149]
Batch 240/700: Discriminator loss = 1.0980504751205444, GAN loss = [2.5961783, 0.96371436, 1.0121734]
Batch 241/700: Discriminator loss = 1.0780571699142456, GAN loss = [2.6487246, 0.9686512, 1.0597625]
Batch 242/700: Discriminator loss = 1.0938773155212402, GAN loss = [2.7055528, 0.9979132, 1.087348]
Batch 243/700: Discriminator loss = 1.083457112312317, GAN loss = [2.8216107, 1.030977, 1.1703285]
Batch 244/700: Discriminator loss = 1.123982548713684, GAN loss = [2.5953996, 0.9921816, 0.9828687]
Batch 245/700: Discriminator loss = 1.1308369636535645, GAN loss = [2.6440334, 0.9773537, 1.0462842]
Batch 246/700: Discriminator loss = 1.0789977312088013, GAN loss = [2.6315193, 1.0151052, 0.99

Batch 331/700: Discriminator loss = 1.0578800439834595, GAN loss = [2.7535102, 1.0647811, 1.0660933]
Batch 332/700: Discriminator loss = 1.0471910238265991, GAN loss = [2.8384907, 1.0735334, 1.1423157]
Batch 333/700: Discriminator loss = 1.087122917175293, GAN loss = [2.6860287, 1.0224361, 1.0409545]
Batch 334/700: Discriminator loss = 1.072474479675293, GAN loss = [2.9626822, 1.0677665, 1.2722809]
Batch 335/700: Discriminator loss = 1.056099534034729, GAN loss = [2.8587875, 1.0581878, 1.1779797]
Batch 336/700: Discriminator loss = 1.080712080001831, GAN loss = [3.0257037, 1.093306, 1.3097442]
Batch 337/700: Discriminator loss = 1.0503767728805542, GAN loss = [2.852411, 1.1024498, 1.127248]
Batch 338/700: Discriminator loss = 1.0839027166366577, GAN loss = [2.8382463, 1.0233111, 1.1921772]
Batch 339/700: Discriminator loss = 1.087592363357544, GAN loss = [2.9838345, 1.0794704, 1.2815608]
Batch 340/700: Discriminator loss = 1.020649790763855, GAN loss = [3.0156536, 1.1316555, 1.2612096]

Batch 424/700: Discriminator loss = 1.1265681982040405, GAN loss = [2.67806, 0.98352724, 1.0710428]
Batch 425/700: Discriminator loss = 1.0909067392349243, GAN loss = [2.8113394, 0.99337506, 1.1944923]
Batch 426/700: Discriminator loss = 1.1368706226348877, GAN loss = [2.7254713, 0.940454, 1.161571]
Batch 427/700: Discriminator loss = 1.1127163171768188, GAN loss = [2.7908075, 1.000056, 1.1673219]
Batch 428/700: Discriminator loss = 1.1230589151382446, GAN loss = [2.7326217, 0.9924549, 1.1167797]
Batch 429/700: Discriminator loss = 1.125461459159851, GAN loss = [2.7737272, 0.98981255, 1.1605852]
Batch 430/700: Discriminator loss = 1.105159044265747, GAN loss = [2.8086605, 1.0182879, 1.1670779]
Batch 431/700: Discriminator loss = 1.1148784160614014, GAN loss = [2.7534473, 0.9879621, 1.1422352]
Batch 432/700: Discriminator loss = 1.0923503637313843, GAN loss = [2.7408679, 1.0406756, 1.0769714]
Batch 433/700: Discriminator loss = 1.1176775693893433, GAN loss = [2.7783406, 1.005446, 1.1497

Batch 520/700: Discriminator loss = 1.1097038984298706, GAN loss = [2.682785, 1.0047812, 1.0551996]
Batch 521/700: Discriminator loss = 1.1252657175064087, GAN loss = [2.660744, 0.980559, 1.0573769]
Batch 522/700: Discriminator loss = 1.1073919534683228, GAN loss = [2.746697, 1.0187751, 1.1051149]
Batch 523/700: Discriminator loss = 1.0661641359329224, GAN loss = [2.7046418, 1.0394716, 1.0423536]
Batch 524/700: Discriminator loss = 1.080053448677063, GAN loss = [2.649813, 1.020181, 1.0068148]
Batch 525/700: Discriminator loss = 1.076124668121338, GAN loss = [2.7670045, 1.0398494, 1.1043391]
Batch 526/700: Discriminator loss = 1.0453249216079712, GAN loss = [2.8380225, 1.0412986, 1.1739148]
Batch 527/700: Discriminator loss = 1.0615390539169312, GAN loss = [2.727809, 1.0481218, 1.0569028]
Batch 528/700: Discriminator loss = 1.0779167413711548, GAN loss = [2.7891772, 1.0315849, 1.1348269]
Batch 529/700: Discriminator loss = 1.0901974439620972, GAN loss = [2.761674, 1.0454298, 1.0934902]


Batch 619/700: Discriminator loss = 1.1150346994400024, GAN loss = [2.6864908, 0.98629016, 1.0790063]
Batch 620/700: Discriminator loss = 1.0864708423614502, GAN loss = [2.7688785, 1.0163921, 1.1313021]
Batch 621/700: Discriminator loss = 1.0871765613555908, GAN loss = [2.7454228, 1.0058708, 1.1183755]
Batch 622/700: Discriminator loss = 1.1034915447235107, GAN loss = [2.7322886, 0.9834591, 1.1276344]
Batch 623/700: Discriminator loss = 1.106684684753418, GAN loss = [2.6820834, 0.97260386, 1.0882674]
Batch 624/700: Discriminator loss = 1.0959856510162354, GAN loss = [2.782425, 0.9837655, 1.1774515]
Batch 625/700: Discriminator loss = 1.0680218935012817, GAN loss = [2.815336, 1.0365252, 1.1575894]
Batch 626/700: Discriminator loss = 1.0694438219070435, GAN loss = [2.7321126, 1.0160774, 1.0948161]
Batch 627/700: Discriminator loss = 1.085098147392273, GAN loss = [2.7414052, 1.0356628, 1.0845379]
Batch 628/700: Discriminator loss = 1.076171875, GAN loss = [2.858462, 1.0360627, 1.2012143]


In [4]:
label_dim

NameError: name 'label_dim' is not defined

In [3]:
# ------------------------------------------------------------------------------
# Plotting
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 2))
for i in range(label_dim):
    im = generator.predict([np.random.uniform(-1, 1, (1, noise_dim)), to_categorical(i, label_dim)])[0].reshape((28, 28))
    plt.subplot(1, label_dim, i+1)
    plt.axis('off')
    plt.imshow(im, cmap='Greys_r')
plt.show()

NameError: name 'label_dim' is not defined

<Figure size 1440x144 with 0 Axes>